In [1]:
import torch

from model import GradKNN
from configs.config import config

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
from utils.other import GradKNNDataloader

data_loader = GradKNNDataloader(num_tasks=6, dataset_name="imagenet-r", load_covariances=True, load_prototypes=False)
model = GradKNN(config, device=device)

for i in range(6):
    X_train, y_train, X_test, y_test, covariances, prototypes = data_loader.get_data(i)
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    acc = (torch.sum((y_test.flatten().to(device) == predictions).int()) / X_test.shape[0] * 100).item()
    print(f"Accuracy: {acc} MY\n")

FeCAM: 81.2191162109375
FeCAM: 76.51376342773438
FeCAM: 72.17646789550781
FeCAM: 71.65015411376953
FeCAM: 69.70014190673828
FeCAM: 67.13153076171875
Accuracy: 81.21910858154297 MY

Accuracy: 76.51376342773438 MY

Accuracy: 72.17646789550781 MY

Accuracy: 71.650146484375 MY

Accuracy: 69.70014190673828 MY

Accuracy: 67.13153076171875 MY

